In [1]:
import keras
import tensorflow
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_video import VideoFrameGenerator

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import keras_metrics as km
import glob
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

In [2]:
data_dir = "./videodata_tmp/"
filename = "ConvLSTM_model_UCF101.h5"
h, w = 224, 224
time_steps = 64

#classes = ["Backstroke", "Breaststroke", "Freestyle"]
classes = [i.split(os.path.sep)[1] for i in glob.glob('./videodata_tmp/*')]
classes.sort()
#print(classes)

#glob_pattern='./videodata/{classname}/*.avi'

#data_aug = ImageDataGenerator(
#    zoom_range = [0.6, 1.2],
#    horizontal_flip = True,
#    rotation_range = 180,
#    width_shift_range = 0,
#    height_shift_range = 0)

#train = VideoFrameGenerator(
#    classes=classes, 
#    glob_pattern=glob_pattern,
#    nb_frames=time_steps,
#    split = 0.3, 
#    shuffle = True,
#    batch_size = 1,
#    target_shape = (w, h),
#    nb_channel = 3,
#    transformation=data_aug,
#    use_frame_cache=True)
#valid = train.get_validation_generator()

In [ ]:

#  Creating frames from videos
def frames_extraction(video_path):
    frames = []
     
    video = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
    while count <= time_steps:
        ret, img = video.read() 
        if ret:
            img = cv2.resize(img, (w, h))
            frames.append(img)
            count += 1
        else:
            print("Defected frame")
            break
    return frames
    #rotate_frames = []
    #for img in frames:
        #rimg = cv2.rotate(img, cv2.ROTATE_180)
        #rotate_frames.append(rimg)

    #return frames, rotate_frames

def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
    print(classes_list)
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
            
            frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            
            if len(frames) == time_steps:
                X.append(frames)
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
             
                
            #if len(frames) == time_steps and len(rotate_frames) == time_steps:
            #    X.append(frames)
            #    y = [0]*len(classes)
            #    y[classes.index(c)] = 1
            #    Y.append(y)
                
            #    X.append(rotate_frames)
            #    y = [0]*len(classes)
            #    y[classes.index(c)] = 1
            #    Y.append(y)
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

X, Y = create_data(data_dir)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=0)


['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth', 'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen', 'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'Hammering', 'HammerThrow', 'HandstandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 'JugglingBalls', 'JumpingJack', 'JumpRope', 'Kayaking', 'Knitting', 'LongJump', 'Lunges', 'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing', 'PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'P

In [15]:
model = Sequential()
# input_shape: (time_steps, map_height, map_width, channels)
model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), input_shape = (time_steps, h, w, 3), return_sequences = True))
model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), return_sequences = True))
#model.add(LeakyReLU(alpha=0.3))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), return_sequences = True))
model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), return_sequences = True))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.summary()

opt = Adam(lr=1e-5)
#opt = SGD(lr=0.0001)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

checkpoint = ModelCheckpoint(filepath=filename,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

stop = EarlyStopping(monitor='loss', patience = 5,
                      verbose=0, mode='auto', baseline=None, 
                      restore_best_weights=False)
callbacks = [checkpoint, stop]

history = model.fit(x = X_train, y = y_train, epochs = 1000, batch_size = 2, shuffle=True, validation_split=0.2, callbacks=callbacks)
#history = model.fit(train, validation_data=valid, verbose=1, epochs=50, shuffle=True, callbacks=callbacks)
print("Training finished.")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_4 (ConvLSTM2D)  (None, 10, 68, 43, 16)    11008     
_________________________________________________________________
conv_lst_m2d_5 (ConvLSTM2D)  (None, 10, 66, 41, 16)    18496     
_________________________________________________________________
activation_2 (Activation)    (None, 10, 66, 41, 16)    0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 10, 66, 41, 16)    64        
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 66, 41, 16)    0         
_________________________________________________________________
conv_lst_m2d_6 (ConvLSTM2D)  (None, 10, 64, 39, 16)    18496     
_________________________________________________________________
conv_lst_m2d_7 (ConvLSTM2D)  (None, 10, 62, 37, 16)   

In [ ]:
#y_train = model.predict(X_train)
#y_val = model.predict(X_test)

#y_train = np.argmax(y_train, axis = 1)
#y_val = np.argmax(y_val, axis = 1)

#print("Train: "+str(y_train))
#print("Validation: "+str(y_val))